In [27]:
import numpy as np
import pandas as pd
import openpyxl
import re as re

In [28]:
df1=pd.read_excel('manuf_data.xlsx',engine='openpyxl')
df2=pd.read_excel('retailer_data.xlsx',engine='openpyxl')

In [29]:
df1.head()

,prod_id,Description
0,355,Mexifiz Shampoo Intensive Repair 400ml
1,356,Mexifiz Shampoo Intensive Repair 400ml
2,381,Cbv Shampoo Moisture Rich 900ml
3,382,Cbv Shampoo Moisture Rich 900ml
4,426,Cbvery Hairspray Extra Hold 400ml


In [30]:
df2.head()

,ret_prod_id,Product_Description
0,188,Oddaced Give Me Moisture Shampoo 250ml
1,334,Cbvery Freeze Hold Hairspray 400ml
2,328,Cbvery Firm Hold Hairspray 400ml
3,203,Oddaced Ultimate Hold Hairspray 400ml
4,324,Cbvery Extra Hold Hairspray 400ml


In [31]:
df1['Description'] = df1['Description'].str.rstrip('ml')   #Remove some words
df2['Product_Description'] = df2['Product_Description'].str.rstrip('ml')   #Remove some words

In [32]:
#Make New Column as Quantity

In [33]:
#Find Product name
def find_word(text):
    word = re.findall(r"[a-zA-z]+",text)
    return " ".join(word)
df1['manuf_product']=df1['Description'].apply(lambda x: find_word(x))
print("\Extracting 'Product Name' from Description:")

\Extracting 'Product Name' from Description:


In [34]:
#Find Product name
def find_word(text):
    word = re.findall(r"[a-zA-z]+",text)
    return " ".join(word)
df2['retailer_product']=df2['Product_Description'].apply(lambda x: find_word(x))
print("\Extracting 'Product Name' from Description:")

\Extracting 'Product Name' from Description:


In [35]:
df1.drop(['Description'], axis=1, inplace = True)
df2.drop(['Product_Description'], axis=1, inplace = True)

In [36]:
df1.head()
#df2.head()

,prod_id,manuf_product
0,355,Mexifiz Shampoo Intensive Repair
1,356,Mexifiz Shampoo Intensive Repair
2,381,Cbv Shampoo Moisture Rich
3,382,Cbv Shampoo Moisture Rich
4,426,Cbvery Hairspray Extra Hold


In [37]:
df1.manuf_product.str.split(expand=True).stack().value_counts().head(10)

Cbv         193
Shampoo     127
Bot         123
Co          101
Oddaced      62
Mexifiz      51
Fiur         37
Colour       33
Hold         31
Moisture     30
dtype: int64

In [38]:
df2.retailer_product.str.split(expand=True).stack().value_counts().head(10)

Mexifiz        104
Shampoo         69
for             67
Cbvery          60
Good            50
Parrot          47
Hair            46
Conditioner     44
Anti            42
Fiur            42
dtype: int64

# Used Name Entity Recognition To Find Product Name 

In [39]:
import spacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
%matplotlib inline

In [40]:
df2.head(1)

,ret_prod_id,retailer_product
0,188,Oddaced Give Me Moisture Shampoo


In [41]:
tweetText = df1['manuf_product']
from nltk.tokenize import word_tokenize

tweetText = tweetText.apply(word_tokenize)
tweetText.head()

0    [Mexifiz, Shampoo, Intensive, Repair]
1    [Mexifiz, Shampoo, Intensive, Repair]
2           [Cbv, Shampoo, Moisture, Rich]
3           [Cbv, Shampoo, Moisture, Rich]
4         [Cbvery, Hairspray, Extra, Hold]
Name: manuf_product, dtype: object

In [42]:
tweetText = df2['retailer_product']
from nltk.tokenize import word_tokenize

tweetText2 = tweetText.apply(word_tokenize)
tweetText2.head()

0    [Oddaced, Give, Me, Moisture, Shampoo]
1         [Cbvery, Freeze, Hold, Hairspray]
2           [Cbvery, Firm, Hold, Hairspray]
3      [Oddaced, Ultimate, Hold, Hairspray]
4          [Cbvery, Extra, Hold, Hairspray]
Name: retailer_product, dtype: object

In [43]:
tokens= []
for i in tweetText:
    for j in i:
        tokens.append(j)  
tokens2 = []
for i in tweetText2:
    for j in i:
        tokens2.append(j)        

In [44]:
# # Function to convert   
# def listToString(s):  
    
#     # initialize an empty string 
#     str1 = " " 
    
#     # return string   
#     return (str1.join(s)) 
# a = listToString(tokens)

# doc = nlp(a)
# a
# #doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the mobile phone market and ordered the company to alter its practices')
# print([(X.text, X.label_) for X in doc.ents])

In [45]:
tokens = nlp(''.join(str(df1.manuf_product.tolist())))
#tokens
tokens2 = nlp(''.join(str(df2.retailer_product.tolist())))

In [46]:
#extract entity
items = [x.text for x in tokens.ents]
Counter(items).most_common(10)

[("Cbv Co Moisture Rich Bot'", 8),
 ("'Cbv Shampoo Deep Cleansing'", 7),
 ("Cbv Shampoo Keratin'", 7),
 ("'Cbv Shampoo Colour Revitalising'", 7),
 ("'Cbv Shampoo Moisture Rich'", 5),
 ("Cbv Co Repair protect Bot'", 5),
 ("'Cbv Shampoo Salon Silk'", 5),
 ("'Cbv Shampoo Hour Body'", 5),
 ("'Cbv Co Hour Body Bot'", 5),
 ("'Cbv Co Keratin Bot'", 4)]

In [47]:
#extract entity
items = [x.text for x in tokens2.ents]
Counter(items).most_common(10)

[("Deodorant Aerosol'", 8),
 ("'Twicebeaut Muru Muru Butter", 6),
 ('Deodorant Spray', 5),
 ("Mexifiz Intensive Body Lotion'", 4),
 ('Deodorant Roll', 4),
 ('Parrot Africa', 4),
 ("Replenish Shampoo'", 3),
 ('Shampoo', 3),
 ('Mexifiz Invisible Dry', 3),
 ("Replenish Conditioner'", 3)]

In [51]:
#Extracting Named Entity
product_list = []
for ent in tokens.ents:
    if ent.label_ == 'ORG':
        product_list.append(ent.text)
product_counts = Counter(product_list).most_common(100)
df_product = pd.DataFrame(product_counts, columns = ['text', 'count'])

In [53]:
#Extracting Named Entity
product_list2 = []
for ent in tokens2.ents:
    if ent.label_ == 'ORG':
        product_list2.append(ent.text)
product_counts = Counter(product_list).most_common(100)
df_product2 = pd.DataFrame(product_counts, columns = ['text', 'count'])


In [54]:
count = 0
for i in df_product['text']:
    for j in df_product2['text']:
        if i == j:
            count = count + 1
count            

28

# 28 Products are best possible match on both side. 
